# WD in python with fortran

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FormatStrFormatter

from IPython.display import display, Math
import subprocess
import sys
import re
import statistics
import pandas as pd
from threading import Timer

* Some useful constants

In [2]:
c = 299792458;            #[m/s]
UA = 149597870700;        #[m] #According https://aas.org/posts/story/2013/08/report-2012-iau-xxviii-general-assembly
pi = np.pi;

#According https://www.iau.org/static/resolutions/IAU2015_English.pdf 
G = 6.67408*10**(-11);    # 2014 CODATA 6.67408 (±0.00031)×10−11m3kg−1s−2
G_gauss = (2*pi)**2       # [year, AU, M_Sun] 
R_Sun = 6.957*10**(8);    #[m]
L_Sun = 3.827*10**(26)    #[W]
M_Sun = 1.9891*10**(30);  #[Kg]
R_Ear = 6.3781*10**(6);   #[m]
R_Jup = 7.1492*10**(7);   #[m]
M_Jup = 1.89813*10**(27); #[Kg]
T_Sun = 5778;

- Define initial values

In [3]:
# data for TIC74386000
obj = 'TIC74386000'

mode = 2
p = prv = 33.62089659
ep = eprv = 2490.00686273
T1t = 1.86304
T10_err1 = 200
T10_err2 = 200
ft = 1

alb1 = 0.5
alb2 = 0.5
hl = 1.0
cl = 1.0
zt = 0.00
ext = 0.00
the = 0.00

flux_s = 1.00

F1 = F2 = 1
dperrt = 0.00
at = 100.00
vgat = 78.00
dpct = 0.00

qt = 0.36
it = 89.50
T2t = 1.80
pot1t = 31.00
pot2t = 27.40

ecct = 0.4415
perrt = 2.0886
phase0 = -0.0529

In [4]:
def countLines(file):
    linicio = 0
    lfim = 0
    with open(file, "r") as l:
        for number, lines in enumerate(l):
            if re.findall(r'\s*JD\s+Phase', lines):
                linicio=number+1
            if linicio != 0 and re.findall(r'^$', lines):
                lfim = number - linicio
                break
    l.close()
    return linicio, lfim

def countLines2(file):
    linicio = 0
    lfim = 0
    with open(file, "r") as l:
        for number, lines in enumerate(l):
            if re.findall(r'\s*Star\s+M/Msun', lines):
                linicio=number
            if linicio != 0 and re.findall(r'^$', lines):
                lfim = number - linicio
                break
    l.close()
    return linicio, lfim

def countLines3(file):
    linicio = 0
    lfim = 0
    with open(file, "r") as l:
        for number, lines in enumerate(l):
            if re.findall(r'\s*Y\s+Sky', lines):
                linicio=number+1
            if linicio != 0 and re.findall(r'^$', lines):
                lfim = number - linicio
                break
    l.close()
    return linicio, lfim

def countLines4(file):
    linicio = 0
    lfim = 0
    with open(file, "r") as l:
        for number, lines in enumerate(l):
            if re.findall(r'\s*star\s+r pole', lines):
                linicio=number+2
            if linicio != 0 and re.findall(r'^$', lines):
                lfim = linicio+2
                break
    l.close()
    return linicio, lfim

# - Run wilson & devinney with fortran

In [5]:
model_dim = []

n_frames = 60
hjd_dim = [1/n_frames]

for i in range(n_frames):
    #dimensions
    
    np.savetxt('hjd.dat', hjd_dim)

    lixo = []
    lixo2 = '{:1d} {:1d} {:1d} {:1d} {:1d} {:1d} {:1d} {:1d} {:2d} {:2d} {:1d} {:1d} {:1d} {:1d} {:6d}'.format(5, 2, 2, 0, 0, 1, 1, 0, -3, -3, 1, 1, 1, 0, 0)
    lixo.append(lixo2)
    lixo2 = '{:1d}{:15.6f}{:17.10f}{:14.6f}{:10.4f}{:8.5f}{:3d}{:11.4f}{:2d}{:11.0f}'.format(2, ep, p, 0.0000, phase0, 00.0000, 1, 0, 2, 138472375)
    lixo.append(lixo2)  
    lixo2 = '{:14.6f}{:15.6f}{:13.6f}{:12.6f}{:12.6f}{:12.6f}{:12.6f}{:10.4f}{:2d}{:8.4f}'.format(30979.347628, 30979.347628, 00000.10000, 00.00000, 001.000000, 000.010000, 000.250000, 0.7500, 1, 1.0140)
    lixo.append(lixo2)  
    lixo2 = '{:2d}{:2d}{:2d}{:2d}{:4d}{:4d}{:13.6f}{:14.6f}{:8.5f}{:8.2f}'.format(mode, 0, 1, 1, 30, 30, perrt, dperrt, the, 1.00)
    lixo.append(lixo2)
    lixo2 = '{:6.5f}{:12.6f}{:10.4f}{:10.4f}{:10.4f}{:9.3f}{:7.3f}{:7.3f}{:7.2f}{:10.4f}{:10.4f}'.format(ecct, at, F1, F2, vgat, it, 1.00, 1.00, -0.50, 1.000, 1.0000)
    lixo.append(lixo2)
    lixo2 = '{:7.4f} {:7.4f}{:7.3f}{:7.3f}{:13.6f}{:13.6f}{:13.6f}{:7.3f}{:7.3f}{:7.3f}{:7.3f}{:8.5f}'.format(T1t, T2t, alb1, alb2, pot1t, pot2t, qt, 0.500, 0.600, 0.172, 0.163, dpct)
    lixo.append(lixo2)
    lixo2 = '{:12.6f}{:14.7f}{:11.5f}{:9.6f}{:10.7f}{:17.8f}'.format(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    lixo.append(lixo2)  
    lixo2 = '{:3d}{:13.7f}{:13.7f}{:7.3f}{:7.3f}{:7.3f}{:7.3f}{:12.4f}{:11.4f}{:8.3f}{:8.4f}{:10.6f}{:8.4f}{:12.5f}'.format(28, hl, cl, 0.260, 0.225, -0.827, -0.723, 0.00000, 0.0000, zt, flux_s, 0.55000, ext, 0.378)
    lixo.append(lixo2)
    lixo2 = '{:9.5f}{:9.5f}{:9.5f}{:9.5f}{:14.5f}{:14.5f}{:14.5f}{:14.5f}'.format(0300.000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000)
    lixo.append(lixo2)
    lixo2 = '{:9.5f}{:9.5f}{:9.5f}{:9.5f}{:14.5f}{:14.5f}{:14.5f}{:14.5f}'.format(0300.000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000)
    lixo.append(lixo2)
    lixo2 = '{:3.0f}.'.format(150.)
    lixo.append(lixo2)
    lixo2 = '{:1.0f} '.format(9)
    lixo.append(lixo2)

    lixo3 = open('lcin.active', "w")
    for line in lixo:
        # write line to output file
        lixo3.write(line)
        lixo3.write("\n")
    lixo3.close()

    #run W&D with fortran
    subprocess.run(["./lc.e"])

    linicio, lfim = countLines3('lcout.active')
    model_dim.append(np.loadtxt('lcout.active', skiprows=linicio, max_rows=lfim))

    hjd_dim = [hjd_dim[0] + 1/n_frames]

In [6]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
fig, ax = plt.subplots()

def animate(t):
    plt.cla()
    plt.xlim(-1.1, 1.1)
    plt.ylim(-0.75, 0.75)
    
    plt.plot(0, 0, '+C3', ms=3, markeredgewidth=1)
    plt.plot(model_dim[t][:,0], model_dim[t][:,1], 'k.', ms=2)    

matplotlib.animation.FuncAnimation(fig, animate, frames=n_frames)

In [7]:
# Save animation as video (if required)
anim = FuncAnimation(fig, animate, frames=n_frames, interval=2)
anim.save('3d_{}.gif'.format(obj))

MovieWriter ffmpeg unavailable; using Pillow instead.
